Projeto de Transfer Learning em Python
O projeto consiste em aplicar o método de Transfer Learning em uma rede de Deep Learning na linguagem Python no ambiente COLAB.  

In [ ]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [ ]:
!wget --no-check-certificate "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip" -O "/home/cats-and-dogs.zip"

local_zip = '/home/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/home')
zip_ref.close()

--2025-02-09 22:42:25--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.210.240.213, 2600:1408:9000:6ac::317f, 2600:1408:9000:68a::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.210.240.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘/home/cats-and-dogs.zip’

/home/cats-and-dogs 100%[===================>] 786.67M  79.4MB/s    in 9.4s    

2025-02-09 22:42:35 (83.4 MB/s) - ‘/home/cats-and-dogs.zip’ saved [824887076/824887076]



In [ ]:
print(len(os.listdir('/home/PetImages/Cat/')))
print(len(os.listdir('/home/PetImages/Dog/')))

12501
12501


In [ ]:
try:
  os.mkdir('/home/cats-v-dogs')
  os.mkdir('/home/cats-v-dogs/training')
  os.mkdir('/home/cats-v-dogs/testing')
  os.mkdir('/home/cats-v-dogs/training/cats')
  os.mkdir('/home/cats-v-dogs/training/dogs')
  os.mkdir('/home/cats-v-dogs/testing/cats')
  os.mkdir('/home/cats-v-dogs/testing/dogs')
except OSError as e:
  print("Error creating director: {e}")
  pass

Definição da Rede Neural


In [ ]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  files = []
  for filename in os.listdir(SOURCE):
    file = SOURCE + filename
    if os.path.getsize(file) > 0:
        files.append(filename)
    else:
        print(filename + " is zero length, so ignoring.")

  training_length = int(len(files) * SPLIT_SIZE)
  testing_length = int(len(files) - training_length)
  shuffled_set = random.sample(files, len(files))
  training_set = shuffled_set[0:training_length]
  testing_set = shuffled_set[-testing_length:]

  for filename in training_set:
    this_file = SOURCE + filename
    destination = TRAINING + filename
    copyfile(this_file, destination)

  for filename in testing_set:
    this_file = SOURCE + filename
    destination = TESTING + filename
    copyfile(this_file, destination)

CAT_SOURCE_DIR = "/home/PetImages/Cat/"
TRAINING_CATS_DIR = "/home/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/home/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/home/PetImages/Dog/"
TRAINING_DOGS_DIR = "/home/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/home/cats-v-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [ ]:
print(len(os.listdir('/home/cats-v-dogs/training/cats/')))
print(len(os.listdir('/home/cats-v-dogs/training/dogs/')))
print(len(os.listdir('/home/cats-v-dogs/testing/cats/')))
print(len(os.listdir('/home/cats-v-dogs/testing/dogs/')))

11250
11250
1250
1250


In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=0.001), loss='binary_crossentropy', metrics=['acc'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
TRAINING_DIR = "/home/cats-v-dogs/training"
train_datagen = ImageDataGenerator(rescale=1.0/255.)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=100,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/home/cats-v-dogs/testing"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=100,
                                                              class_mode='binary',
                                                              target_size=(150, 150))

Found 22499 images belonging to 2 classes.
Found 2499 images belonging to 2 classes.


In [ ]:
history = model.fit(train_generator, epochs=15, steps_per_epoch=20, validation_data=validation_generator, validation_steps=6)

Epoch 1/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step - acc: 0.5621 - loss: 0.6907 - val_acc: 0.4983 - val_loss: 0.7286
Epoch 2/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - acc: 0.5415 - loss: 0.6963 - val_acc: 0.5117 - val_loss: 0.6938
Epoch 3/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - acc: 0.5665 - loss: 0.6948 - val_acc: 0.5717 - val_loss: 0.6677
Epoch 4/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - acc: 0.6029 - loss: 0.6672 - val_acc: 0.6400 - val_loss: 0.6516
Epoch 5/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 83s 4s/step - acc: 0.6533 - loss: 0.6405 - val_acc: 0.6400 - val_loss: 0.6436
Epoch 6/15
11/20 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - acc: 0.6766 - loss: 0.6282

/usr/local/lib/python3.11/dist-packages/PIL/TiffImagePlugin.py:949: UserWarning: Truncated File Read
  warnings.warn(str(msg))


20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - acc: 0.6721 - loss: 0.6314 - val_acc: 0.6883 - val_loss: 0.6086
Epoch 7/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 53s 3s/step - acc: 0.6715 - loss: 0.6237 - val_acc: 0.6767 - val_loss: 0.6178
Epoch 8/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - acc: 0.6399 - loss: 0.6279 - val_acc: 0.7033 - val_loss: 0.6169
Epoch 9/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - acc: 0.7064 - loss: 0.5955 - val_acc: 0.6983 - val_loss: 0.5858
Epoch 10/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - acc: 0.6856 - loss: 0.5799 - val_acc: 0.7133 - val_loss: 0.5706
Epoch 11/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - acc: 0.6830 - loss: 0.5871 - val_acc: 0.7033 - val_loss: 0.5584
Epoch 12/15
 5/20 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - acc: 0.7509 - loss: 0.5217

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


20/20 ━━━━━━━━━━━━━━━━━━━━ 21s 954ms/step - acc: 0.7562 - loss: 0.5221 - val_acc: 0.7050 - val_loss: 0.5629
Epoch 13/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 109s 3s/step - acc: 0.7010 - loss: 0.5802 - val_acc: 0.7350 - val_loss: 0.5296
Epoch 14/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - acc: 0.7065 - loss: 0.5416 - val_acc: 0.6817 - val_loss: 0.5693
Epoch 15/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 82s 4s/step - acc: 0.7167 - loss: 0.5565 - val_acc: 0.7483 - val_loss: 0.5125
